In [2]:
from oasis import SocialAgent, AgentGraph
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from oasis import ActionType, LLMAction, ManualAction

In [3]:
model =  ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O,
    model_config_dict={"temperature": 0.0},
)

In [4]:
available_actions = [
        ActionType.LIKE_POST,
        ActionType.DISLIKE_POST,
        ActionType.CREATE_POST,
        ActionType.CREATE_COMMENT,
        ActionType.LIKE_COMMENT,
        ActionType.DISLIKE_COMMENT,
        ActionType.SEARCH_POSTS,
        ActionType.SEARCH_USER,
        ActionType.TREND,
        ActionType.REFRESH,
        ActionType.DO_NOTHING,
        ActionType.FOLLOW,
        ActionType.MUTE,
    ]

In [5]:
from dataclasses import dataclass, asdict, field
import warnings
from camel.prompts import TextPrompt

@dataclass
class UserInfo:
    occupation: str = "学生"
    age: int = 20
    hobbies: list = field(default_factory=list)
    residence: str = "東京"
    user_profile: str = "毎日２キロ走ってます。"
    recsys_type: str = "twitter"
    is_controllable: bool = False

    def to_custom_system_message(self, user_info_template: TextPrompt) -> str:
        required_keys = user_info_template.key_words
        profile = asdict(self)

        info_keys = set(profile.keys())
        missing = required_keys - info_keys
        extra = info_keys - required_keys

        if missing:
            raise ValueError(
                f"Missing required keys in UserInfo.profile: {missing}")
        if extra:
            warnings.warn(f"Extra keys not used in UserInfo.profile: {extra}")

        return user_info_template.format(**profile)

    def to_system_message(self) -> str:
        if self.recsys_type != "reddit":
            return self.to_twitter_system_message()
        else:
            return self.to_reddit_system_message()

    def to_twitter_system_message(self) -> str:
        hobbies_str = ", ".join(self.hobbies)
        description = (
            f"You are a {self.age} year old {self.occupation} living in {self.residence}. "
            f"Your hobbies are {hobbies_str}. "
            f"{self.user_profile}"
        )

        system_content = f"""
# OBJECTIVE
You're a Twitter user, and I'll present you with some posts. After you see the posts, choose some actions from the following functions.

# SELF-DESCRIPTION
Your actions should be consistent with your self-description and personality.
{description}

# RESPONSE METHOD
Please perform actions by tool calling.
        """

        return system_content


In [6]:
import json

def generare_x_agent_graph(
    profile_path,
    model,
    available_actions
):
    with open(profile_path, 'r', encoding='utf-8') as f:
        agent_info = json.load(f)
    agent_graph = AgentGraph()
    for agent_id in range(len(agent_info)):
        profile = {
            "nodes": [],
            "edges": [],
            "other_info": {},
        }
        
        profile["other_info"]["user_profile"] = agent_info[agent_id]

        user_info = UserInfo(
            occupation = agent_info[agent_id]["occupation"],
            age = agent_info[agent_id]["age"],
            hobbies = agent_info[agent_id]["hobbies"],
            residence = agent_info[agent_id]["residence"],
            user_profile = agent_info[agent_id]["user_profile"],
            recsys_type = "twitter"
        )

        agent = SocialAgent(
            agent_id=agent_id,
            user_info=user_info,
            model=model,
            agent_graph=agent_graph,
            available_actions=available_actions,
        )

        agent_graph.add_agent(agent)
    
    return agent_graph
            


In [7]:
agent_graph = generare_x_agent_graph(
    profile_path="../data/persona/persona.json",
    model=model,
    available_actions=available_actions
)

In [8]:
agent_graph = _

In [9]:
agent_graph

''

In [12]:
import os
import oasis
# Define the path to the database
db_path = "../data/twitter_simulation.db"

# Delete the old database
if os.path.exists(db_path):
    os.remove(db_path)

# Make the environment
env = oasis.make(
    agent_graph=agent_graph,
    platform=oasis.DefaultPlatformType.TWITTER,
    database_path=db_path,
)

# Run the environment
await env.reset()

actions_1 = {}
actions_1[env.agent_graph.get_agent(0)] = [
    ManualAction(action_type=ActionType.CREATE_POST,
                    action_args={"content": "Hello, world!"}),
    ManualAction(action_type=ActionType.CREATE_COMMENT,
                    action_args={
                        "post_id": "1",
                        "content": "Welcome to the OASIS World!"
                    })
]
actions_1[env.agent_graph.get_agent(1)] = ManualAction(
    action_type=ActionType.CREATE_COMMENT,
    action_args={
        "post_id": "1",
        "content": "I like the OASIS world."
    })
await env.step(actions_1)

actions_2 = {
    agent: LLMAction()
    for _, agent in env.agent_graph.get_agents()
}

# Perform the actions
await env.step(actions_2)

# Close the environment
await env.close()

db_path ../data/twitter_simulation.db


AttributeError: 'str' object has no attribute 'get_agents'